# Import Demand

In [49]:
def intervalo_horario(horario):
    return '07:00' < horario < '09:00'
def intervalo_horario_am_pm(horario):
    return '7:00:00 AM' < horario < '9:00:00 AM'
def encode(s):
    return s.encode('latin1').decode('unicode_escape')

### Read jsons

In [50]:
import os
import json

In [51]:
files = []
dir_path = 'data/disciplinas/'
for path in os.listdir(dir_path):
    if os.path.isfile(os.path.join(dir_path, path)):
        files.append(path)

In [52]:
len(files)

5205

In [53]:
disciplinas = []
for file in files:
    with open(dir_path + file, 'r+') as f:
        content = json.loads(f.read())
        disciplinas.append(content)

In [54]:
disciplinas[1000]

{'unidade': 'Escola Politécnica',
 'departamento': 'Eng Telecomunicações e Controle',
 'campus': 'São Paulo',
 'codigo': 'PTC3502',
 'nome': 'Estágio Supervisionado',
 'creditos_aula': 1,
 'creditos_trabalho': 6,
 'objetivos': 'Realização de um estágio relacionado ao futuro exercício profissional, com a supervisão de um professor do curso.',
 'programa_resumido': 'Estágio em empresa, universidade ou instituição de pesquisa.',
 'turmas': [{'codigo': '2022250',
   'inicio': '15/08/2022',
   'fim': '21/12/2022',
   'tipo': 'Teórica',
   'horario': [{'dia': 'sab',
     'inicio': '09:20',
     'fim': '10:10',
     'professores': ['Cristiano Magalhaes Panazio']}],
   'vagas': {'Obrigatória': {'vagas': 35,
     'inscritos': 8,
     'pendentes': 0,
     'matriculados': 8,
     'grupos': {'EP - Engenharia Elétrica - EC3 (Telecomunicações)': {'vagas': 35,
       'inscritos': 6,
       'pendentes': 0,
       'matriculados': 6}}},
    'Extracurricular': {'vagas': 2,
     'inscritos': 0,
     'pend

### Filter Cuaso

In [274]:
fora_cuaso = ['Escola de Artes, Ciências e Humanidades', 'Escola de Enfermagem', 'Faculdade de Direito', 'Faculdade de Medicina',
              'Faculdade de Saúde Pública', 'Faculdade de Saúde Pública e Faculdade de Ciências Farmacêuticas', 'Escola de Enfermagem de Ribeirão Preto',
              'Museu de Zoologia']
def is_disciplina_cuaso(disciplina, fora_cuaso):
    if disciplina['campus'] != 'São Paulo':
        return False
    if disciplina['unidade'] in fora_cuaso:
        return False
    return True

In [275]:
disciplinas_cuaso = []
for disciplina in disciplinas:
    if is_disciplina_cuaso(disciplina, fora_cuaso):
        disciplinas_cuaso.append(disciplina)
len(disciplinas_cuaso)

2352

In [276]:
unidades_matrusp = {}
for disciplina in disciplinas_cuaso:
    unidades_matrusp[disciplina['unidade']] = 1
unidades_matrusp

{'Instituto Oceanográfico': 1,
 'Escola Politécnica': 1,
 'Escola de Comunicações e Artes': 1,
 'Instituto de Matemática e Estatística': 1,
 'Faculdade de Filosofia, Letras e Ciências Humanas': 1,
 'Instituto de Química': 1,
 'Instituto de Ciências Biomédicas': 1,
 'Instituto de Geociências': 1,
 'Instituto de Biociências': 1,
 'Instituto de Psicologia': 1,
 'Faculdade de Medicina Veterinária e Zootecnia': 1,
 'Faculdade de Ciências Farmacêuticas': 1,
 'Escola de Educação Física e Esporte': 1,
 'Faculdade de Educação': 1,
 'Instituto de Física': 1,
 'Instituto de Astronomia, Geofísica e Ciências Atmosféricas': 1,
 'Faculdade de Odontologia': 1,
 'Faculdade de Arquitetura e Urbanismo': 1,
 'Instituto de Relações Internacionais': 1,
 'Faculdade de Economia, Administração, Contabilidade e Atuária': 1,
 'Faculdade de Medicina, Instituto de Ciências Biomédicas, Instituto de Química e Instituto de Biociências': 1,
 'Instituto de Estudos Brasileiros': 1,
 'Museu de Arqueologia e Etnologia': 1

### Get Classes

In [277]:
aulas = []
for i, disciplina in enumerate(disciplinas_cuaso):
    for turma in disciplina['turmas']:
        total_matriculados = 0
        for k, vaga in turma['vagas'].items():
            total_matriculados += vaga['matriculados']
            
        if turma['horario'] != None:
            for horarios in turma['horario']:
                aula = {}
                aula['indice_disc'] = i
                aula['codigo'] = disciplina['codigo']
                aula['codigo_turma'] = turma['codigo']
                aula['unidade'] = disciplina['unidade']
                aula['dia'] = horarios['dia']
                aula['horario_inicio'] = horarios['inicio']
                aula['total_matriculados'] = total_matriculados
                aulas.append(aula)

### Filter Classes

In [278]:
aulas_intervalo_horario = []
for aula in aulas:
    if intervalo_horario(aula['horario_inicio']):
        aulas_intervalo_horario.append(aula)

In [279]:
agrupamento_dia_instituto = {}
agrupamento_dia_instituto['seg'] = {}
agrupamento_dia_instituto['ter'] = {}
agrupamento_dia_instituto['qua'] = {}
agrupamento_dia_instituto['qui'] = {}
agrupamento_dia_instituto['sex'] = {}
agrupamento_dia_instituto['sab'] = {}
agrupamento_dia_instituto['dom'] = {}
for aula in aulas_intervalo_horario:
    if aula['unidade'] not in agrupamento_dia_instituto[aula['dia']]:
        agrupamento_dia_instituto[aula['dia']][aula['unidade']] = 0
    agrupamento_dia_instituto[aula['dia']][aula['unidade']] += aula['total_matriculados']

### Read research

In [280]:
import pandas as pd
import datetime as dt

In [281]:
pesquisa_perguntas = pd.read_csv('./data/disciplinas/forms/resposta_pesq.csv', delimiter=';')
pesquisa_perguntas['timestamp'] = list(map(lambda x: dt.datetime.strptime(x, '%m/%d/%Y %H:%M:%S'), pesquisa_perguntas['Carimbo de data/hora']))
pesquisa_perguntas = pesquisa_perguntas[pesquisa_perguntas['timestamp'] >= dt.datetime(2022, 10, 7, 0, 0, 0)]
len(pesquisa_perguntas)

4395

In [282]:
pesquisa = pd.DataFrame()
pesquisa['vinculo'] = pesquisa_perguntas['Qual o seu vínculo com a USP?']
pesquisa['instituto'] = pesquisa_perguntas['A qual instituto você está associado?']


# ida
pesquisa['utiliza_circular_ida'] = pesquisa_perguntas['Quantas vezes por semana você utiliza o circular para entrar na USP?'] >= 2
pesquisa['linhas_ida'] = pesquisa_perguntas['Qual das linhas abaixo você utiliza neste trajeto?']
pesquisa['ponto_embarque_ida'] = pesquisa_perguntas['Por qual ponto você normalmente embarca?']
pesquisa['ponto_desembarque_ida'] = pesquisa_perguntas['Em qual ponto de ônibus você normalmente desembarca?']
pesquisa['horario_ida'] = pesquisa_perguntas['Que horas você normalmente chega na USP?']


# volta
pesquisa['utiliza_circular_volta'] = pesquisa_perguntas['Quantas vezes por semana você utiliza o circular para sair da USP?'] >= 2
pesquisa['linhas_volta'] = pesquisa_perguntas['Qual das linhas abaixo você utiliza neste trajeto?.1']
pesquisa['ponto_embarque_volta'] = pesquisa_perguntas['Em qual o ponto de ônibus você embarca para sair da USP?']
pesquisa['ponto_desembarque_volta'] = pesquisa_perguntas['Por qual ponto você normalmente desembarca?']
pesquisa['horario_volta'] = pesquisa_perguntas['Que horas você normalmente chega no ponto de ônibus para sair da USP?']

pesquisa.head()

,vinculo,instituto,utiliza_circular_ida,linhas_ida,ponto_embarque_ida,ponto_desembarque_ida,horario_ida,utiliza_circular_volta,linhas_volta,ponto_embarque_volta,ponto_desembarque_volta,horario_volta
136,Aluno de graduação,Instituto de Matemática e Estatística (IME),True,"8012, 8022",P3,FAU II,7:54:00 AM,True,"8012, 8022",FAU II,Portão da Vila Indiana,7:10:00 PM
137,Aluno de graduação,Faculadae de Arquitetura e Urbanismo (FAU),True,"8012, 8032",Estação Butantã,FEA,1:10:00 PM,True,"8012, 8022",FAU II,Estação Butantã,10:30:00 PM
138,Aluno de graduação,Instituto de Matemática e Estatística (IME),True,"8012, 8032",Estação Butantã,FEA,7:15:00 PM,True,8022,Rua do Matão 0,Estação Butantã,10:40:00 PM
139,Aluno de pós-graduação,"Faculdade de Filosofia, Letras e Ciências Huma...",True,8012,PTREM,Geografia/História,7:25:00 PM,True,8012,Geografia/História,PTREM,9:40:00 PM
140,Aluno de graduação,Instituto de Relações internacionais (IRI),False,NaN,NaN,NaN,NaN,False,NaN,NaN,NaN,NaN


### Filter students

In [283]:
pesquisa[['vinculo']].value_counts()

vinculo                                                        
Aluno de graduação                                                 2880
Aluno de pós-graduação                                              752
Funcionário                                                         644
Docente                                                              36
docente                                                              22
Professor                                                            21
professor                                                             8
Ex-aluno                                                              5
Professor                                                             5
Docente                                                               4
Professor titular                                                     1
aprimoramento                                                         1
Pós-doutorando                                                        1


In [284]:
pesquisa[['vinculo', 'utiliza_circular_ida']].value_counts()

vinculo                                                          utiliza_circular_ida
Aluno de graduação                                               True                    2532
Aluno de pós-graduação                                           True                     575
Funcionário                                                      True                     369
Aluno de graduação                                               False                    348
Funcionário                                                      False                    275
                                                                                         ... 
Docente                                                          False                      1
Comunidade                                                       True                       1
Aluno e funcionario                                              True                       1
Aluno e Funcionário.                                             Tru

In [285]:
def grad_ou_pos(x):
    return x == 'Aluno de graduação' or x == 'Aluno de pós-graduação'
pesquisa_alunos = pesquisa[list(map(grad_ou_pos, pesquisa['vinculo']))]
pesquisa_alunos[['vinculo']].value_counts()

vinculo               
Aluno de graduação        2880
Aluno de pós-graduação     752
dtype: int64

### Filtro horário e cálculo porcentagem carro por instituto

In [286]:
pesquisa_alunos.head()

,vinculo,instituto,utiliza_circular_ida,linhas_ida,ponto_embarque_ida,ponto_desembarque_ida,horario_ida,utiliza_circular_volta,linhas_volta,ponto_embarque_volta,ponto_desembarque_volta,horario_volta
136,Aluno de graduação,Instituto de Matemática e Estatística (IME),True,"8012, 8022",P3,FAU II,7:54:00 AM,True,"8012, 8022",FAU II,Portão da Vila Indiana,7:10:00 PM
137,Aluno de graduação,Faculadae de Arquitetura e Urbanismo (FAU),True,"8012, 8032",Estação Butantã,FEA,1:10:00 PM,True,"8012, 8022",FAU II,Estação Butantã,10:30:00 PM
138,Aluno de graduação,Instituto de Matemática e Estatística (IME),True,"8012, 8032",Estação Butantã,FEA,7:15:00 PM,True,8022,Rua do Matão 0,Estação Butantã,10:40:00 PM
139,Aluno de pós-graduação,"Faculdade de Filosofia, Letras e Ciências Huma...",True,8012,PTREM,Geografia/História,7:25:00 PM,True,8012,Geografia/História,PTREM,9:40:00 PM
140,Aluno de graduação,Instituto de Relações internacionais (IRI),False,NaN,NaN,NaN,NaN,False,NaN,NaN,NaN,NaN


In [327]:
pesquisa_ida = pesquisa_alunos[pesquisa_alunos['horario_ida'].notna()]
pesquisa_alunos_intervalo = pesquisa_ida[list(map(intervalo_horario_am_pm, pesquisa_ida['horario_ida']))]
pesquisa_alunos_intervalo

,vinculo,instituto,utiliza_circular_ida,linhas_ida,ponto_embarque_ida,ponto_desembarque_ida,horario_ida,utiliza_circular_volta,linhas_volta,ponto_embarque_volta,ponto_desembarque_volta,horario_volta
136,Aluno de graduação,Instituto de Matemática e Estatística (IME),True,"8012, 8022",P3,FAU II,7:54:00 AM,True,"8012, 8022",FAU II,Portão da Vila Indiana,7:10:00 PM
138,Aluno de graduação,Instituto de Matemática e Estatística (IME),True,"8012, 8032",Estação Butantã,FEA,7:15:00 PM,True,8022,Rua do Matão 0,Estação Butantã,10:40:00 PM
139,Aluno de pós-graduação,"Faculdade de Filosofia, Letras e Ciências Huma...",True,8012,PTREM,Geografia/História,7:25:00 PM,True,8012,Geografia/História,PTREM,9:40:00 PM
142,Aluno de graduação,Escola Politécnica (Poli),True,"8022, 8032",Estação Butantã,Poli Metalurgia,7:40:00 AM,True,"8012, 8032",Poli Mecânica,Estação Butantã,3:00:00 PM
143,Aluno de graduação,Escola Politécnica (Poli),True,"8012, 8022, 8032",Estação Butantã,Poli Biênio,8:30:00 AM,True,"8012, 8032",Poli Biênio,Estação Butantã,5:40:00 PM
...,...,...,...,...,...,...,...,...,...,...,...,...
4524,Aluno de graduação,"Faculdade de Filosofia, Letras e Ciências Huma...",True,"8012, 8032",Estação Butantã,Biblioteca Brasiliana,8:00:00 AM,True,"8012, 8032","R. Prof. Mello Moraes, 1473 (localizado entre ...",Estação Butantã,1:10:00 PM
4525,Aluno de graduação,Escola Politécnica (Poli),True,"8012, 8022, 8032",Estação Butantã,Poli Civil,7:30:00 AM,True,"8012, 8022, 8032",Poli Civil,Estação Butantã,5:00:00 PM
4527,Aluno de graduação,Instituto de Biociências (IB),True,"8022, 8032",Estação Butantã,Biociências,7:40:00 AM,True,"8022, 8032",Ponto Paços das Artes,Estação Butantã,10:15:00 PM
4528,Aluno de graduação,Escola de Comunicações e Artes (ECA),True,"8012, 8032",Estação Butantã,CRUSP (Praça da Reitoria),8:10:00 AM,True,8012,ECA,Estação Butantã,6:00:00 PM


In [288]:
def retira_sigla(x):
    index = x.find('(')
    
    if index < 0:
        return x
    
    return x[:index - 1]

def filtra_instituto_pesq(x):
    x = retira_sigla(x)
    
    if x == 'ECA e MAC USP': 
        return 'Escola de Comunicações e Artes'
    elif x == 'FFLCH':
        return 'Faculdade de Filosofia, Letras e Ciências Humanas'
    elif x in ['IPEN', 'IPEN ', 'ipen', 'Instituto de pesquisas energéticas e nucleares', 'Instituto de Pesquisa Energéticas e Nucleares', 'Instituto de Pesquisas Energéticas e Nucleares - IPEN', 'Instituto de pesquisas energéticas e nucleares ', 'Instituto de pesquisas energéticas e nucleares -IPEN', 'Instituo de Pesquisas Energéticas e Nucleares']:
        return 'Instituto de Energia e Ambiente'
    elif x == 'MAC':
        return 'Museu de Arte Contemporânea'
    elif x in ['PRG', 'Prg ', 'Pró-Reitoria de Graduação', 'Pró-Reitoria de graduação', 'Pró-reitoria de graduacao', 'Curso de Ciências Moleculares']:
        return None
    elif x == 'Faculadae de Arquitetura e Urbanismo':
        return 'Faculdade de Arquitetura e Urbanismo'
    elif x == 'Instituo de Astronomia, Geofísica e Ciências Atmosféricas':
        return 'Instituto de Astronomia, Geofísica e Ciências Atmosféricas'
    elif x == 'Faculdade de Economia, Administração e Contabilidade':
        return 'Faculdade de Economia, Administração, Contabilidade e Atuária'
    elif x == 'MAC':
        return 'Museu de Arte Contemporânea'
    elif x == 'Escola de Aplicação da Faculdade de Educação (EA-FEUSP)':
        return 'Faculdade de Educação'
    elif x == 'Instituto de Relações internacionais':
        return 'Instituto de Relações Internacionais'
    else:
        return x
    
pesquisa_alunos_intervalo2 = pesquisa_alunos_intervalo.copy()
pesquisa_alunos_intervalo2['instituto'] = list(map(filtra_instituto_pesq, pesquisa_alunos_intervalo['instituto']))

In [289]:
# Filtra os Nones

pesquisa_alunos_intervalo = pesquisa_alunos_intervalo2
pesquisa_alunos_intervalo = pesquisa_alunos_intervalo[pesquisa_alunos_intervalo['instituto'].notna()]
pesquisa_alunos_intervalo['instituto']

136                 Instituto de Matemática e Estatística
138                 Instituto de Matemática e Estatística
139     Faculdade de Filosofia, Letras e Ciências Humanas
142                                    Escola Politécnica
143                                    Escola Politécnica
                              ...                        
4524    Faculdade de Filosofia, Letras e Ciências Humanas
4525                                   Escola Politécnica
4527                             Instituto de Biociências
4528                       Escola de Comunicações e Artes
4529                             Faculdade de Odontologia
Name: instituto, Length: 1775, dtype: object

In [290]:
def mapeia_cursos_de_fora(x):
    if x in ['Faculdade de Medicina', 'Escola de Enfermagem', 'Faculdade de Saúde Pública']:
        return 'Faculdade de Medicina, Instituto de Ciências Biomédicas, Instituto de Química e Instituto de Biociências'
    elif x == 'Faculdade de Direito':
        return 'Faculdade de Filosofia, Letras e Ciências Humanas'
    elif x == 'MAC':
        return 'Museu de Arte Contemporânea'
    else:
        return x
    
pesquisa_alunos_intervalo2 = pesquisa_alunos_intervalo.copy()
pesquisa_alunos_intervalo2['instituto'] = list(map(mapeia_cursos_de_fora, pesquisa_alunos_intervalo['instituto']))
pesquisa_alunos_intervalo = pesquisa_alunos_intervalo2

In [301]:
pesquisa_alunos_intervalo['instituto'].unique()

array(['Instituto de Matemática e Estatística',
       'Faculdade de Filosofia, Letras e Ciências Humanas',
       'Escola Politécnica', 'Escola de Artes, Ciências e Humanidades',
       'Faculdade de Medicina, Instituto de Ciências Biomédicas, Instituto de Química e Instituto de Biociências',
       'Faculdade de Economia, Administração, Contabilidade e Atuária',
       'Instituto Oceanográfico', 'Escola de Comunicações e Artes',
       'Instituto de Química', 'Instituto de Ciências Biomédicas',
       'Faculdade de Odontologia', 'Instituto de Física',
       'Escola de Educação Física e Esporte',
       'Faculdade de Ciências Farmacêuticas',
       'Instituto de Astronomia, Geofísica e Ciências Atmosféricas',
       'Instituto de Geociências',
       'Faculdade de Medicina Veterinária e Zootecnia',
       'Instituto de Biociências', 'Faculdade de Arquitetura e Urbanismo',
       'Instituto de Psicologia', 'Instituto de Energia e Ambiente',
       'Instituto de Relações Internacionais

### Filter: utiliza circular, butantã vs. p3, horário

In [292]:
porc_circular_por_instituto = pesquisa_alunos_intervalo[['instituto','utiliza_circular_ida']].groupby('instituto').agg('mean').reset_index()

pd.set_option('display.max_rows', len(porc_circular_por_instituto))

In [299]:
for dia in agrupamento_dia_instituto:
    for instituto in agrupamento_dia_instituto[dia]:
        agrupamento_dia_instituto[dia][instituto] *= porc_circular_por_instituto[porc_circular_por_instituto['instituto'] == instituto]['utiliza_circular_ida'].iloc[0]
        agrupamento_dia_instituto[dia][instituto] = int(agrupamento_dia_instituto[dia][instituto])

# Pessoas por instituto que utilizam circular de acordo com a pesquisa
agrupamento_dia_instituto

{'seg': {'Instituto de Geociências': 85,
  'Faculdade de Medicina Veterinária e Zootecnia': 485,
  'Escola de Comunicações e Artes': 430,
  'Escola Politécnica': 1676,
  'Faculdade de Filosofia, Letras e Ciências Humanas': 848,
  'Instituto de Matemática e Estatística': 939,
  'Faculdade de Arquitetura e Urbanismo': 314,
  'Faculdade de Educação': 5,
  'Faculdade de Ciências Farmacêuticas': 101,
  'Faculdade de Economia, Administração, Contabilidade e Atuária': 330,
  'Escola de Educação Física e Esporte': 50,
  'Faculdade de Odontologia': 290,
  'Instituto de Ciências Biomédicas': 154,
  'Instituto de Pesquisas Energéticas e Nucleares': 2,
  'Instituto de Física': 294,
  'Instituto de Química': 157,
  'Instituto Oceanográfico': 63,
  'Faculdade de Medicina, Instituto de Ciências Biomédicas, Instituto de Química e Instituto de Biociências': 47,
  'Instituto de Biociências': 33,
  'Instituto de Psicologia': 82,
  'Instituto de Astronomia, Geofísica e Ciências Atmosféricas': 5},
 'ter': 

In [320]:
df1 = pesquisa_alunos_intervalo[pesquisa_alunos_intervalo['instituto'] == 'Escola Politécnica']
df2 = df1[df1['ponto_desembarque_ida'] == 'CRUSP (Praça da Reitoria)']
df2['ponto_desembarque_ida'].value_counts().iloc[0]

0

In [322]:
def calcula_quantidade_pessoas_ponto(df, instituto, ponto):
    df1 = df[pesquisa_alunos_intervalo['instituto'] == instituto]
    df2 = df1[df1['ponto_desembarque_ida'] == ponto]
    
    if len(df2['ponto_desembarque_ida']) == 0:
        return 0
    
    return df2['ponto_desembarque_ida'].value_counts().iloc[0]

institutos_ponto_rel = {}
for instituto in pesquisa_alunos_intervalo['instituto'].unique():
    institutos_ponto_rel[instituto] = {}
    for ponto in pesquisa_alunos_intervalo['ponto_desembarque_ida'].unique():
        institutos_ponto_rel[instituto][ponto] = calcula_quantidade_pessoas_ponto(pesquisa_alunos_intervalo, instituto, ponto)

In [338]:
import random
def usa_linha(str_linhas, linha_target):
    linhas = str_linhas.split(', ')
    linha_selecionada = linhas[random.randint(0, len(linhas) - 1)]
    return (linha_selecionada == linha_target)

def calcula_quantidade_pessoas_ponto_linha(df, ponto, linha):
    df2 = df[df['ponto_desembarque_ida'] == ponto]
    
    if len(df2['ponto_desembarque_ida']) == 0:
        return 0
    
    df1 = df2[list(map(lambda x: usa_linha(x, linha), df2['linhas_ida']))]
    
    if len(df1['linhas_ida']) == 0:
        return 0
    
    return df1['ponto_desembarque_ida'].value_counts().iloc[0]

dia_hora_ponto_linha = {}

for dia in agrupamento_dia_instituto:
    dia_hora_ponto_linha[dia] = {}
    for hora in [480]:
        dia_hora_ponto_linha[dia][hora] = {}
        for ponto in pesquisa_alunos_intervalo['ponto_desembarque_ida'].unique():
            dia_hora_ponto_linha[dia][hora][ponto] = {}
            for linha in ['8012', '8022', '8032']:
                dia_hora_ponto_linha[dia][hora][ponto][linha] = calcula_quantidade_pessoas_ponto_linha(pesquisa_alunos_intervalo, ponto, linha)

In [345]:
import numpy as np

class NpEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.integer):
            return int(obj)
        if isinstance(obj, np.floating):
            return float(obj)
        if isinstance(obj, np.ndarray):
            return obj.tolist()
        return super(NpEncoder, self).default(obj)
print(json.dumps(dia_hora_ponto_linha, indent=4, cls=NpEncoder))

{
    "seg": {
        "480": {
            "FAU II": {
                "8012": 21,
                "8022": 34,
                "8032": 5
            },
            "FEA": {
                "8012": 99,
                "8022": 49,
                "8032": 110
            },
            "Geografia/Hist\u00f3ria": {
                "8012": 13,
                "8022": 12,
                "8032": 5
            },
            "Poli Metalurgia": {
                "8012": 6,
                "8022": 14,
                "8032": 6
            },
            "Poli Bi\u00eanio": {
                "8012": 57,
                "8022": 59,
                "8032": 58
            },
            "Reitoria/Bancos": {
                "8012": 41,
                "8022": 12,
                "8032": 27
            },
            "Biblioteca Brasiliana": {
                "8012": 84,
                "8022": 30,
                "8032": 75
            },
            "P3": {
                "8012": 32,
            

In [226]:
def is_butanta(x): 
    return x == 'Estação Butantã' or x == 'PTREM'

intervalo_butanta = pesquisa_alunos[pesquisa_alunos['utiliza_circular_ida']]
intervalo_butanta = intervalo_butanta[list(map(intervalo_horario_am_pm, intervalo_butanta['horario_ida']))]
intervalo_butanta = intervalo_butanta[list(map(is_butanta, intervalo_butanta['ponto_embarque_ida']))]
intervalo_butanta

,vinculo,instituto,utiliza_circular_ida,linhas_ida,ponto_embarque_ida,ponto_desembarque_ida,horario_ida,utiliza_circular_volta,linhas_volta,ponto_embarque_volta,ponto_desembarque_volta,horario_volta
138,Aluno de graduação,Instituto de Matemática e Estatística (IME),True,"8012, 8032",Estação Butantã,FEA,7:15:00 PM,True,8022,Rua do Matão 0,Estação Butantã,10:40:00 PM
139,Aluno de pós-graduação,"Faculdade de Filosofia, Letras e Ciências Huma...",True,8012,PTREM,Geografia/História,7:25:00 PM,True,8012,Geografia/História,PTREM,9:40:00 PM
142,Aluno de graduação,Escola Politécnica (Poli),True,"8022, 8032",Estação Butantã,Poli Metalurgia,7:40:00 AM,True,"8012, 8032",Poli Mecânica,Estação Butantã,3:00:00 PM
143,Aluno de graduação,Escola Politécnica (Poli),True,"8012, 8022, 8032",Estação Butantã,Poli Biênio,8:30:00 AM,True,"8012, 8032",Poli Biênio,Estação Butantã,5:40:00 PM
145,Aluno de graduação,Instituto de Matemática e Estatística (IME),True,8012,Estação Butantã,FEA,8:15:00 AM,True,8012,Física,Estação Butantã,11:50:00 AM
...,...,...,...,...,...,...,...,...,...,...,...,...
4520,Aluno de pós-graduação,Museu de Arqueologia e Etnologia (MAE),True,8022,Estação Butantã,Prof Almeida Prado,8:50:00 AM,True,"8012, 8022, 8032",depende,Estação Butantã,3:00:00 PM
4524,Aluno de graduação,"Faculdade de Filosofia, Letras e Ciências Huma...",True,"8012, 8032",Estação Butantã,Biblioteca Brasiliana,8:00:00 AM,True,"8012, 8032","R. Prof. Mello Moraes, 1473 (localizado entre ...",Estação Butantã,1:10:00 PM
4525,Aluno de graduação,Escola Politécnica (Poli),True,"8012, 8022, 8032",Estação Butantã,Poli Civil,7:30:00 AM,True,"8012, 8022, 8032",Poli Civil,Estação Butantã,5:00:00 PM
4527,Aluno de graduação,Instituto de Biociências (IB),True,"8022, 8032",Estação Butantã,Biociências,7:40:00 AM,True,"8022, 8032",Ponto Paços das Artes,Estação Butantã,10:15:00 PM


### Transform to stations

TODO:
- pegar uma linha randomica
- linhas_ida e ponto_desembarque_ida

### Add employees and professors